### 微调前

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Request 547adf6b-245e-403f-bb1c-068101f7ee1b: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/config.json (authenticated: False)
Request 9dc58b92-8540-424c-8a63-14b42d10d1d1: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/adapter_config.json (authenticated: False)
Request bf8e8859-612f-4c32-b74a-a70b80c17218: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/configuration_chatglm.py (authenticated: False)
Request 135bd83f-c0c5-4524-ba7a-111a93f6112e: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/modeling_chatglm.py (authenticated: False)
Request aadd14ad-bcde-4312-8bc5-54e6c83cc0a2: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/adapter_config.json (authenticated: False)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Request 6de826d0-3c66-4136-8479-3ec7f0c618e7: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/generation_config.json (authenticated: False)


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

Request 62037a96-a3b9-415b-bebc-d720b85ad8e2: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/tokenizer_config.json (authenticated: False)
Request 786309d8-8479-4f1f-bc1f-5a2c0c9d7af5: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/tokenization_chatglm.py (authenticated: False)


In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图之一，其符号为阳，代表天、云、雷、司法等。乾卦是由两个阴爻夹一个阳爻构成，象征阳刚之气，具有领导、创造、坚定、果敢等特性。在八宫图中，乾卦位于西北方，与事业、权力、组织、法律等方面有关。乾卦的五行属性为阳，属于火，因此，乾卦也与热情、主动、积极、进取等有关。在易经中，乾卦的卦辞为“元、亨、利、贞”，表示具有正始、正谋、正道、正果的品质。


In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，其卦象征诉讼、争端、诉讼仲裁等，代表矛盾、纷争和不和。讼卦由两个阳爻夹一个阴爻构成，象征阳刚之师，具有刚强、刚毅、坚定等特性。在八宫图中，讼卦位于西南方，与法律、争端、诉讼、仲裁等方面有关。

讼卦的五行属性为阳，属于火，因此，讼卦也与热情、主动、积极、进取等有关。在易经中，讼卦的卦辞为“元、亨、利、贞”，表示具有正始、正谋、正道、正果的品质。然而，由于诉讼往往伴随着争议和损失，因此，讼卦也提醒人们要慎重处理争端，避免不必要的损失和矛盾。


### QLoRA微调后

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240401_110946"

peft_model_path = f"/root/autodl-tmp//THUDM/chatglm3-6b-epoch1-20240401_110946"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(handmake))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，也是八宫图说、易经、易学中最基本、最重要的卦象之一。乾卦代表着天、强、积极、刚健、力量、行动等。乾卦是由两个阴爻夹一个阳爻构成，象征天宇的广阔、刚强、积极和茂盛。

乾卦的卦辞是“元、亨、利、永、久、健”，意味着乾卦具有永恒的、普遍的、顺利的、茂盛的、刚健的和行动的力量。在八宫图说中，乾卦主宰着北方，与事业、努力、积极、刚健、力量、行动等有关，同时也与皇帝、领导、父亲等有关。

在易经中，乾卦的阳爻代表阳刚之气，阴爻代表阴柔之气。乾坤二元阴阳配合，构成世界上的一切事物和现象。乾卦所代表的阳刚之气，是推动事物发展、进步的主要力量。因此，乾卦具有强烈的前进性、创造性和行动性。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(handmake))-20240401_110946）：
[gMASK]sop 解释下乾卦是什么？
乾卦是八卦之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、云、雷等自然现象，也代表着刚强、积极、行动力等含义。在八宫图（后天八卦）中，乾卦位于北方，与事业、努力、坚定等有关。在五行中，乾卦对应于木，代表着生长、茂盛、积极向上等含义。乾卦的意义在于鼓励人们保持积极向上的心态，努力前进，克服困难，实现自己的目标。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
《周易》中的讼卦（又称法卦、商卦）是八卦之一，位于乾卦和坤卦之间。它是八卦中唯一一个由两个卦相组合而成的卦象。讼卦的卦象是由乾卦和坤卦各下一爻组成的，乾卦代表天、刚、明、强制等，坤卦则代表地、顺、承载、阴等。两卦组合在一起，意味着阳刚与阴柔相互对抗、协商、协调，以达到和解、和谐、平衡的状态。

讼卦的意义非常丰富，它既表示诉讼、争端、争斗等纷争现象，也象征着通过和解、协调、沟通等手段化解矛盾、解决问题。在这个卦象中，阳刚之力量与阴柔之力量相互融合，最终可以达成一种和谐共处、互相支持的状态。因此，讼卦教导我们在处理问题和矛盾时要保持中庸之道，既不能过于刚烈，也不能过于柔顺，要寻求合适的平衡点。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(handmake))-20240401_110946）：
[gMASK]sop 周易中的讼卦是什么？
周易中的讼卦是八卦之一，它的卦象是由两个离字叠加而成，表示诉讼、争端、诉讼等概念。讼卦的意义是解决纷争、和解矛盾，强调通过沟通、协商、妥协等和平方式来解决争端，而不是通过武力、诉讼等强制性手段。

讼卦的卦辞描述了诉讼的过程和结果，其中“初”表示诉讼的开始， “终”表示诉讼的结束， “损”表示通过损失、牺牲来解决问题，“益”表示通过协商、妥协来解决问题。

周易中的讼卦也反映了社会的现实情况，人们在使用诉讼解决争端时，应该遵循公平、公正、公开的原则，尽量避免使用武力、威胁等不道德手段。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是《易经》中的第五卦，由两个阴爻夹一个阳爻构成，象征着阳柔阴刚之义。师卦的卦辞为：“天风鼎，元吉，亨。”意味着“天风鼎”这个卦具有元吉之兆，预示着行动将取得成功。

师卦主要代表教育、教导、引导等含义。在《易经》中，师卦所象征的教育不仅仅是知识的传授，更重要的是道德的引导和行为的规范。它反映了人们在学习、教育、领导等方面所体现出的智慧、德行和能力。

此外，师卦还有一个重要的象征意义，即“师”，在古代指代教师，后来泛指教育者。因此，师卦也可以理解为对教育者的赞美和尊敬。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(handmake))-20240401_110946）：
[gMASK]sop 师卦是什么？
师卦是《易经》中的一个卦象，由两个阴爻夹一个阳爻构成，象征着军队或力量。师卦的含义包括：军队、力量、领导、教导等。在《易经》中，师卦通常被认为是一个阳刚之卦，具有积极向上、勇敢担当的品质。同时，师卦也提醒人们要善于教导、引导他人，以达到和谐团结的目的。
